## Benchmark Point in Polygon

Some realistic test cases of enriching polygons with point data and vice versa, using Uber's NYC Trip Data and census blocks.

Inspired by this post:

https://www.linkedin.com/posts/mbforr_python-sql-spatialsql-activity-6834177271616589824-ScB-

In [ ]:
con = omnisci_connect()
con

#### Spatial Join Timing Test

##### Enriching uber points with census ids

In [ ]:
q = "SELECT "
q +=    "trips.trip_datetime, trips.base, trips.omnisci_geo, "
q +=    "census.STATE_FIPS, census.CNTY_FIPS, census.STCOFIPS, census.TRACT, census.BLKGRP, census.FIPS  "
q += "FROM "
q +=    "uber_nyc_trips_geo as trips, us_census_bg as census "
q += "WHERE "
q +=   f"ST_CONTAINS(census.omnisci_geo, trips.omnisci_geo)"
q

In [ ]:
t0 = time()
con.con.execute(q)
t1 = time()
print(f"Spatial enrichment of 4.5m points in memory with geo column took {t1-t0} seconds")

In [ ]:
ctas = f"CREATE TABLE IF NOT EXISTS uber_nyc_census_enriched_geo AS ({q})"
ctas

In [ ]:
t0 = time()
con.con.execute(ctas)
t1 = time()
print(f"Spatial enrichment of 4.5m points with materialization took {t1-t0} seconds")

Using raw longitude and latitude columns

In [ ]:
trip_points = "ST_SETSRID(ST_POINT(trips.longitude, trips.latitude),4326)"
q = "SELECT "
q +=    "trips.trip_datetime, trips.base, "
q +=   f"{trip_points} as omnisci_geo, "
q +=    "census.STATE_FIPS, census.CNTY_FIPS, census.STCOFIPS, census.TRACT, census.BLKGRP, census.FIPS  "
q += "FROM "
q +=    "uber_nyc_trips as trips, us_census_bg as census "
q += "WHERE "
q +=   f"ST_CONTAINS(census.omnisci_geo, {trip_points})"
q

In [ ]:
t0 = time()
con.con.execute(q)
t1 = time()
print(f"Spatial enrichment of 4.5m points in memory with raw columns took {t1-t0} seconds")

In [ ]:
con.con.execute("DROP TABLE IF EXISTS uber_nyc_census_enriched")

In [ ]:
ctas = f"CREATE TABLE IF NOT EXISTS uber_nyc_census_enriched AS ({q})"

In [ ]:
t0 = time()
con.con.execute(ctas)
t1 = time()
print(f"Spatial enrichment of 4.5m points with materialization took {t1-t0} seconds")

#### Summarization on Block Group Geometries

In [ ]:
trip_points = "ST_SETSRID(ST_POINT(trips.longitude, trips.latitude),4326)"
q = "SELECT "
q +=    "count(*) as trip_count, count(distinct trips.base) as base_count, "
q +=   f"sample(census.omnisci_geo) as omnisci_geo, "
q +=    "sample(census.STATE_FIPS) as STATE_FIPS, "
q +=    "sample(census.CNTY_FIPS) as CNTY_FIPS, "
q +=    "sample(census.STCOFIPS) as STCOFIPS, "
q +=    "sample(census.TRACT) as TRACT, "
q +=    "sample(census.BLKGRP) as BLKGRP, "
q +=    "census.FIPS  "
q += "FROM "
q +=    "uber_nyc_trips as trips, us_census_bg as census "
q += "WHERE "
q +=   f"ST_CONTAINS(census.omnisci_geo, {trip_points})"
q += "GROUP BY "
q +=   "census.FIPS"
q

In [ ]:
t0 = time()
con.con.execute(q)
t1 = time()
print(f"Spatial enrichment of census block polygons from 4.5m points in memory took {t1-t0} seconds")

In [ ]:
con.con.execute("DROP TABLE IF EXISTS nyc_census_bg_uber_enriched")

In [ ]:
ctas = f"CREATE TABLE IF NOT EXISTS nyc_census_bg_uber_enriched AS ({q})"

In [ ]:
t0 = time()
con.con.execute(ctas)
t1 = time()
print(f"Spatial enrichment of census blocks with 4.5m points with materialization took {t1-t0} seconds")